<a href="https://colab.research.google.com/github/yingwang/image_analysis/blob/main/Image_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image comparison in Sharpness, Luminance, Color and Noise level

## Imports

In [ ]:
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow
import dlib
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import ndimage
from skimage.metrics import structural_similarity as ssim

## Config

In [ ]:
img1_filename = "image1.png" # @param ["image1.png"] {allow-input: true}
img2_filename = "image2.png" # @param ["image2.png"] {allow-input: true}
compare_faces_only = False # @param {type:"boolean"}



## Read files

In [ ]:
drive.mount('/content/drive')
input_dir = "/content/drive/MyDrive/input_images/"
output_dir = "/content/drive/MyDrive/output_images/"

os.makedirs(output_dir, exist_ok=True)

img1_path = os.path.join(input_dir, img1_filename)
img2_path = os.path.join(input_dir, img2_filename)
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)
if img1 is None or img2 is None:
  print("Error: One or both images failed to load.")

## Crop

In [ ]:
def align_and_resize_faces(img1, img2):
    """Align and resize faces using facial landmarks"""
    # Initialize dlib face detector and landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(os.path.join(input_dir, 'shape_predictor_68_face_landmarks.dat')) # Model download required

    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    faces1 = detector(gray1)
    faces2 = detector(gray2)

    if len(faces1) > 0 and len(faces2) > 0:
        shape1 = predictor(gray1, faces1[0])
        shape2 = predictor(gray2, faces2[0])

        face1_aligned = align_face(img1, shape1)
        face2_aligned = align_face(img2, shape2)

        final_size = (min(face1_aligned.shape[1], face2_aligned.shape[1]),
                      min(face1_aligned.shape[0], face2_aligned.shape[0]))

        return (cv2.resize(face1_aligned, final_size),
                cv2.resize(face2_aligned, final_size))
    else:
        print("No faces detected")
        return None, None

def align_face(img, shape):
    """Align the face using landmarks"""
    left_eye = (np.mean([shape.part(36).x, shape.part(39).x]), np.mean([shape.part(36).y, shape.part(39).y]))
    right_eye = (np.mean([shape.part(42).x, shape.part(45).x]), np.mean([shape.part(42).y, shape.part(45).y]))

    d_y = right_eye[1] - left_eye[1]
    d_x = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(d_y, d_x))
    center = (img.shape[1] // 2, img.shape[0] // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

    left = min([shape.part(i).x for i in range(17)])
    right = max([shape.part(i).x for i in range(17)])
    top = min(shape.part(19).y, shape.part(24).y)
    bottom = shape.part(8).y
    cropped = rotated[top:bottom, left:right]

    return cropped

In [ ]:
def align_and_crop(img1, img2):
    """Align and crop to the common area with the same size"""
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=50))
    matches = flann.knnMatch(des1, des2, k=2)

    good = [m for m, n in matches if m.distance < 0.7 * n.distance]

    if len(good) >= 4:
        # Calculate homography matrix
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # Calculate the coordinates of the transformed corners of image 1
        h, w = img1.shape[:2]
        corners = np.float32([[0,0], [0,h], [w,h], [w,0]]).reshape(-1,1,2)
        transformed_corners = cv2.perspectiveTransform(corners, H)

        # Calculate the boundaries of the common area
        x_min = max(0, int(min(transformed_corners[:,0,0])))
        y_min = max(0, int(min(transformed_corners[:,0,1])))
        x_max = min(img2.shape[1], int(max(transformed_corners[:,0,0])))
        y_max = min(img2.shape[0], int(max(transformed_corners[:,0,1])))

        # Calculate the cropping dimensions
        crop_width = x_max - x_min
        crop_height = y_max - y_min

        # Calculate the cropping area corresponding to img1
        M_inv = np.linalg.inv(H)
        img1_corners = cv2.perspectiveTransform(
            np.float32([[x_min,y_min], [x_min,y_max], [x_max,y_max], [x_max,y_min]]).reshape(-1,1,2),
            M_inv
        )
        x1_min = max(0, int(np.min(img1_corners[:,0,0])))
        y1_min = max(0, int(np.min(img1_corners[:,0,1])))
        x1_max = min(w, int(np.max(img1_corners[:,0,0])))
        y1_max = min(h, int(np.max(img1_corners[:,0,1])))

        cropped1 = img1[y1_min:y1_max, x1_min:x1_max]
        cropped2 = img2[y_min:y_max, x_min:x_max]

        final_size = (min(cropped1.shape[1], cropped2.shape[1]),
                     min(cropped1.shape[0], cropped2.shape[0]))

        return (cv2.resize(cropped1, final_size),
               cv2.resize(cropped2, final_size))

    else:  # Use center cropping when there are insufficient features
        h = min(img1.shape[0], img2.shape[0])
        w = min(img1.shape[1], img2.shape[1])
        y1 = (img1.shape[0]-h)//2
        x1 = (img1.shape[1]-w)//2
        y2 = (img2.shape[0]-h)//2
        x2 = (img2.shape[1]-w)//2

        return img1[y1:y1+h, x1:x1+w], img2[y2:y2+h, x2:x2+w]

## Analysis

In [ ]:
def color_analysis(img1, img2):
    img1_lab = cv2.cvtColor(img1, cv2.COLOR_BGR2LAB)
    img2_lab = cv2.cvtColor(img2, cv2.COLOR_BGR2LAB)

    l1, a1, b1 = cv2.split(img1_lab)
    l2, a2, b2 = cv2.split(img2_lab)
    b1_rgb, g1_rgb, r1_rgb = cv2.split(img1)
    b2_rgb, g2_rgb, r2_rgb = cv2.split(img2)

    l_diff = cv2.absdiff(l1, l2)
    lab_color_diff = np.sqrt((a1 - a2)**2 + (b1 - b2)**2).astype(np.uint8)

    r_diff = cv2.absdiff(r1_rgb, r2_rgb)
    g_diff = cv2.absdiff(g1_rgb, g2_rgb)
    b_diff = cv2.absdiff(b1_rgb, b2_rgb)

    bgr_diff = cv2.merge([b_diff, g_diff, r_diff])
    return l_diff, lab_color_diff, bgr_diff, r_diff, g_diff, b_diff

def sharpness_analysis(img1, img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sharpness1 = cv2.Laplacian(gray1, cv2.CV_64F).var()
    sharpness2 = cv2.Laplacian(gray2, cv2.CV_64F).var()

    sharp_diff = cv2.absdiff(
        cv2.convertScaleAbs(cv2.Laplacian(gray1, cv2.CV_64F)),
        cv2.convertScaleAbs(cv2.Laplacian(gray2, cv2.CV_64F))
    )
    return sharp_diff, sharpness1, sharpness2

def denoise_analysis(img1, img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Calculate high-frequency energy
    fft1 = np.fft.fft2(gray1)
    fft_shift1 = np.fft.fftshift(fft1)
    magnitude1 = 20 * np.log(np.abs(fft_shift1))

    fft2 = np.fft.fft2(gray2)
    fft_shift2 = np.fft.fftshift(fft2)
    magnitude2 = 20 * np.log(np.abs(fft_shift2))

    noise_diff = cv2.absdiff(magnitude1.astype(np.uint8), magnitude2.astype(np.uint8))
    return noise_diff

def visualize_diff(base_img, diff_map, title, threshold=30):
    _, thresh = cv2.threshold(diff_map, threshold, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    vis_img = base_img.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(vis_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(vis_img, f"{title} Diff", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    return vis_img

## Visualize

In [ ]:
def print_analysis(aligned_img1, aligned_img2, l_diff, bgr_diff, sharp_diff, noise_diff, sharp1, sharp2):
    def to_float(x):
        return float(x.mean() if isinstance(x, np.ndarray) else x)

    # Luminance
    avg_b = [to_float(np.mean(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))) for img in [aligned_img1, aligned_img2]]
    brighter = "(Image 1 > Image 2)" if avg_b[0] > avg_b[1] else "(Image 2 > Image 1)"
    brightness_diff = to_float(np.mean(l_diff))

    # Color
    rgb_diffs = [
        to_float(np.mean(bgr_diff[2])),  # Red
        to_float(np.mean(bgr_diff[1])),  # Green
        to_float(np.mean(bgr_diff[0]))   # Blue
    ]

    # Sharpness
    sharp_rel = "Image 1 > Image 2" if sharp1 > sharp2 else "Image 2 > Image 1"

    # Noise
    noise_values = [to_float(noise_diff[0]), to_float(noise_diff[1])]
    noise_compare = "Image 1 cleaner" if noise_values[0] < noise_values[1] else "Image 2 cleaner"

    print(f"""
    Brightness: {avg_b[0]:.1f} vs {avg_b[1]:.1f} {brighter}
    Sharpness: {sharp1:.1f} vs {sharp2:.1f} ({sharp_rel})
    Noise: {noise_values[0]:.1f} vs {noise_values[1]:.1f} ({noise_compare})

    Color:
      Red:   {rgb_diffs[0]:+.1f} ({'Image 1 > Image 2' if rgb_diffs[0] > 0 else 'Image 2 > Image 1'})
      Green: {rgb_diffs[1]:+.1f} ({'Image 1 > Image 2' if rgb_diffs[1] > 0 else 'Image 2 > Image 1'})
      Blue:  {rgb_diffs[2]:+.1f} ({'Image 1 > Image 2' if rgb_diffs[2] > 0 else 'Image 2 > Image 1'})
        """)

In [ ]:
def analyze_and_visualize(img1, img2, compare_faces_only):
    if compare_faces_only:
        aligned_img1, aligned_img2 = align_and_resize_faces(img1, img2)
    else:
        aligned_img1, aligned_img2 = align_and_crop(img1, img2)

    combined_aligned = np.hstack((aligned_img1, aligned_img2))
    cv2.imwrite('combined.png', combined_aligned)
    cv2_imshow(combined_aligned)
    cv2.waitKey(0)

    l_diff, lab_color_diff, rgb_diff, r_diff, g_diff, b_diff = color_analysis(aligned_img1, aligned_img2)
    sharp_diff, sharp1, sharp2 = sharpness_analysis(aligned_img1, aligned_img2)
    noise_diff = denoise_analysis(aligned_img1, aligned_img2)

    visualizations = {
    "Brightness": visualize_diff(aligned_img1, l_diff, "Brightness", threshold=30),
    "Sharpness": visualize_diff(aligned_img1, sharp_diff, "Sharpness", threshold=25),
    "Denoise": visualize_diff(aligned_img1, noise_diff, "Denoise", threshold=20),
    "LAB Color": visualize_diff(aligned_img1, lab_color_diff, "LAB Color", threshold=20),
    "Red Channel": visualize_diff(aligned_img1, r_diff, "Red Channel", threshold=30),
    "Green Channel": visualize_diff(aligned_img1, g_diff, "Green Channel", threshold=30),
    "Blue Channel": visualize_diff(aligned_img1, b_diff, "Blue Channel", threshold=30),
    }

    combined_visuals = np.vstack((
    np.hstack((visualizations["Brightness"], visualizations["LAB Color"])),
    np.hstack((visualizations["Sharpness"], visualizations["Denoise"])),
    ))

    cv2.imwrite('combined_diff.png', combined_visuals)
    cv2_imshow(combined_visuals)
    cv2.waitKey(0)

    l_diff, lab_color_diff, bgr_diff, r_diff, g_diff, b_diff = color_analysis(aligned_img1, aligned_img2)
    sharp_diff, sharp1, sharp2 = sharpness_analysis(aligned_img1, aligned_img2)
    noise_diff = denoise_analysis(aligned_img1, aligned_img2)

    print_analysis(aligned_img1, aligned_img2,
                  l_diff, bgr_diff,
                  sharp_diff, noise_diff, sharp1, sharp2)

In [ ]:
analyze_and_visualize(img1, img2, compare_faces_only)